<img src="./images/DLI_Header.png" style="width: 400px;">

# 美国手语数据集的图像分类

在本节中，您将使用不同的数据集[American Sign Language](http://www.asl.gs/)中手语字母的图像，执行在上一节中观察到的数据准备、模型创建和模型训练的步骤。

## 目标

在完成本节时，您将能够：
* 准备图像数据进行训练
* 创建并编译用于图像分类的简单模型
* 训练图像分类模型并观察结果

## 手语数据集
我们将使用[美国手语字母表](https://www.kaggle.com/datamunge/sign-language-mnist)图像，该字母表包含 26 个字母。其中有两个字母（j 和 z）涉及一些动作，因此已从本次训练数据集中排除。

<img src="./images/asl.png" style="width: 600px;">

[Kaggle](http://www.kaggle.com) 网站提供了此数据集，此网站内容丰富，您还可在其中查找多个数据集和其他深度学习资源。除了数据集之外，内容贡献者还发布了类似于这些 notebook 的“内核”，并向您展示了如何训练模型和探索数据。如果您正在寻找新的深度学习项目以做入门之用，推荐您访问 Kaggle 网站。该网站还会在其组织的竞赛中附加一些数据集，您可以参与其中，在训练高精度模型方面与他人一较高下。

## 加载数据集

我们将引导您加载ASL数据集，因为它无法像MNIST一样通过Keras获得。在本小节的最后，您将获得您所熟悉的`x_train`，`y_train`，`x_test`和`y_test`变量。

## 读入数据
上一个练习中我们直接从 Keras 内下载数据。与此不同的是，手语数据集采用 CSV 格式。如果您以前使用过电子表格，可能会熟悉 CSV（逗号分隔值）。本质上说，它只是一个由行和列组成的网格，且顶部带有标签。

为加载和处理数据，我们将使用一个名为 Pandas 的库，这是一款可供加载和处理数据的高性能工具。我们将 CSV 文件读入名为 Dataframe 的格式，这是 Pandas 存储数据网格所采用的格式。

In [ ]:
import pandas as pd

Pandas有一个`read_csv`方法，该方法将读取csv文件，并返回一个数据帧（Dataframe）：

In [ ]:
train_df = pd.read_csv("asl_data/sign_mnist_train.csv")
test_df = pd.read_csv("asl_data/sign_mnist_test.csv")

### 探索数据

再来观察一下我们的数据。我们可以使用 `head` 函数来抓取数据集的前几行。如您所见，每一行都是一个具有`label`列的图像，还有784个值代表该图像中的每个像素值，就像MNIST数据集一样。请注意，当前标签是数值，而不是字母：

In [ ]:
train_df.head()

### 提取标签

与MNIST一样，我们希望将训练和测试标签存储在`y_train`和`y_test`变量中。 在这里，我们创建这些变量，然后从不再需要它们的原始数据帧中删除标签：

In [ ]:
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

### 提取图像

与MNIST一样，我们希望将训练和测试图像存储在`x_train`和`x_test`变量中。 我们现在创建这些变量：

In [ ]:
x_train = train_df.values
x_test = test_df.values

### 总结训练和测试数据

如您所见，我们现在有27,455张784像素的图像用于训练...

In [ ]:
x_train.shape

...及其相应的标签：

In [ ]:
y_train.shape

我们有7,172张图片用于测试...

In [ ]:
x_test.shape

...及其相应的标签：

In [ ]:
y_test.shape

### 数据可视化

为了可视化图像数据，我们将再次使用matplotlib库。您无需担心此可视化的细节，但是如果您愿意，可以稍后再了解有关[matplotlib](https://matplotlib.org/)的更多信息。请注意，我们必须将数据从其当前的784像素的1D形状重构为28x28像素的2D形状才能合理呈现图像：

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,40))

num_images = 20
for i in range(num_images):
    row = x_train[i]
    label = y_train[i]
    
    image = row.reshape(28,28)
    plt.subplot(1, num_images, i+1)
    plt.title(label, fontdict={'fontsize': 30})
    plt.axis('off')
    plt.imshow(image, cmap='gray')

## 练习：对图像数据进行归一化

正如我们对MNIST数据集所做的那样，我们将对图像数据进行归一化，这意味着它们的像素值将不是介于0到255之间：

In [ ]:
x_train.min()

In [ ]:
x_train.max()

...而应为0到1之间的浮点值。请使用以下单元格来完成次项工作。如果卡住了，请查看下面的答案。

In [ ]:
# TODO: Normalize x_train and x_test.

### 解答

单击下面的'...'以显示答案。

```python
x_train = x_train / 255
x_test = x_test / 255
```

现在我们可以显示图像了，但我们的标签却是数字的形式，而非以字母表中的字母来表示的。目前这没多大影响，因为我们的模型能够将数字作为预测目标来处理。就像上一个练习一样，我们将采用这些数字并将其转换为由 0 和 1 组成的列表，以便使预测容易一些。

## 练习：标签的分类编码

正如我们对MNIST数据集所做的那样，我们将对标签进行分类编码。回想一下，您可以使用`keras.utils.to_categorical`方法来实现此目的，只需向其传递您希望编码的值，以及您希望的类别数。请在下面的单元格中完成您的工作。我们已导入`keras`并为您设置了类别数（24）。

In [ ]:
import tensorflow.keras as keras
num_classes = 24

In [ ]:
# TODO: Categorically encode y_train and y_test.

### 解答

单击下面的'...'以显示答案。

```python
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
```

## 练习：构建模型

您已经对训练和测试图像进行归一化了，并将训练和测试的标签进行了分类编码。数据已准备就绪。

您将练习构建一个序列化的模型，就像上次建立模型时一样：
* 具有密集输入层。该层应包含512个神经元，使用`relu`激励函数，并期望输入图像的形状为(784，)。
* 具有512个神经元的第二个密集连接层，使用`relu`激励函数。
* 密集输出层，神经元的数量等于类别数，使用`softmax`激励函数。

请在下面的单元格中完成您的工作，创建一个`model`变量来存储您构建的模型。我们已经导入了Keras的`Sequential`模型类和`Dense`层类，以帮助您入门。如果需要帮助，可单击下方的三个点来显示答案。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# TODO: build a model following the guidelines above.

### 答案

单击下面的'...'以显示答案。

In [ ]:
model = Sequential()
model.add(Dense(units = 512, activation='relu', input_shape=(784,)))
model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = num_classes, activation='softmax'))

## 总结模型

运行下面的单元以显示您刚刚创建的模型的摘要：

In [ ]:
model.summary()

## 编译模型

我们将使用与之前相同的选项[多分类交叉熵](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) 作为损失函数来编译模型，且想要衡量模型的准确率。多分类交叉熵反映出如下事实：我们试图拟合出诸多类别中的一类。

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

注意，编译模型时可以选择不同的`优化器`，上面的编译使用了默认的优化器`RMSProp`。

## 练习：训练模型

使用你的模型的`fit`方法，用您创建的训练和测试图像和标签将模型训练20个周期（epochs）：

In [ ]:
# TODO: Train the model for 20 epochs.

### 答案

单击下面的'...'以显示答案。

```python
model.fit(x_train, y_train,
                    epochs=20,
                    verbose=1,
                    validation_data=(x_test, y_test))
```

## 讨论：发生了什么？
我们可以看到训练集上的准确率已经相当高，但验证准确率却没有那么高。您认为此处发生了什么？先思考片刻再查看下一部分的内容。

这是一个典型的例子，即模型学习了对训练集里的数据进行分类，但对于尚未采用的新数据进行分类的表现却并不理想。从本质上讲，这是由于模型只是在训练中记忆数据，而非真正获得对该问题（数据分类）的理解。这是一个常见的问题，称为过拟合。在接下来的两个讲座中，我们将讨论过拟合以及解决此问题的一些方法。

## 总结

在本节中，您构建了自己的神经网络来执行图像分类。尽管还有改进的余地（我们很快会讲如何改进），但您做得不错。 恭喜！

到这里为止，您应该对加载数据（包括标签）、准备数据、创建模型以及使用准备好的数据训练模型的过程有所了解了。

### 清理显存
在进入下一节内容前，请先执行以下单元，以清理 GPU 显存。转至下一 notebook 之前需要执行此操作。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

现在，您已经建立了几个虽然基本但还是有效的模型，您将开始学习更复杂的模型，包括*卷积神经网络*。

请继续下一节：[*使用CNN的ASL*](./03_asl_cnn.ipynb)。